Explanação básica do exemplo apresentado no página [MNIST For ML Beginners](https://www.tensorflow.org/get_started/mnist/beginners).

Esta é apenas uma descrição livre dos pontos relacioandos na página, não tem o proposito de aprofundar no assunto de "Machine Learning".
Para mais detalhes é aconcelhavem ler o site original.

Para a excusão deste código é importante fazer o download dos arquivos do [MNIST](http://yann.lecun.com/exdb/mnist/) e descompactá-los em **`<PASTA DO NOTEBOOK>/MNIST_data`.**

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
%matplotlib inline

Faz a leitura do arquivo contendo as imagens de Treino

In [ ]:
f = open('MNIST_data/train-images.idx3-ubyte', 'rb')    

Este tipo é criado para inverter a ordem de interpretação do campo.
São lidos quatro bytes (x00 x00 x80 x03), como padrão são interpretados de traz para frente, isto é, (x03 x80 x00 x00), resultando no número 50855936, mas é necessário que esta ordem seja invertida para gerar o resultado esperado, então é criado um novo tipo de dado que não faz a inversão quando da interpretação. Resultando no número 2051 que era o esperado.

In [ ]:
dt = np.dtype(np.uint32).newbyteorder('>')

In [ ]:
magic = np.frombuffer(f.read(4), dtype=dt)[0]

In [ ]:
if magic != 2051:
    raise ValueError('Invalid magic number %d in MNIST image file: %s' % (magic, f.name))

In [ ]:
num_images = np.frombuffer(f.read(4), dtype=dt)[0]
rows = np.frombuffer(f.read(4), dtype=dt)[0]
cols = np.frombuffer(f.read(4), dtype=dt)[0]
buf = f.read(rows * cols * num_images)
data = np.frombuffer(buf, dtype=np.uint8)
train_images = data.reshape(num_images, rows, cols, 1)
f.close()

In [ ]:
first_image = train_images[0].copy().reshape((28,28))
first_image = first_image.astype(np.uint8)
plt.imshow(first_image, cmap='Greys')

Faz a leitura do arquivo de rotulos (labels)

In [ ]:
f = open('MNIST_data/train-labels.idx1-ubyte', 'rb')
magic = np.frombuffer(f.read(4), dtype=dt)[0]
if magic != 2049:
  raise ValueError('Invalid magic number %d in MNIST label file: %s' % (magic, f.name))
num_items = np.frombuffer(f.read(4), dtype=dt)[0]
buf = f.read(num_items)
labels = np.frombuffer(buf, dtype=np.uint8)
f.close()

**Entendendo melhor o conjunto de comando abaixo.**

Primerio é atribuido o total de labels.
Em seguida é criado uma lista com o total de itens coletados acima vezes 10, isso porque meu conjunto é de 10 classes (números de 0 a 10), isso para depois no "flat" ficar um número por "linha".
É criada uma matriz de zeros que será a base para a saida.
Por fim é executado o "flat", neste ponto é atribuido 1 a todas as posições indicadas na lista (index_offset + labels). O comando "Ravel", não tem uma função, é mais uma segurança.

In [ ]:
num_labels = labels.shape[0]
index_offset = np.arange(num_labels) * 10
labels_one_hot = np.zeros((num_labels, 10))
labels_one_hot.flat[index_offset + labels.ravel()] = 1
train_labels = labels_one_hot

Carregar as imagens de teste

In [ ]:
f = open('MNIST_data/t10k-images.idx3-ubyte', 'rb')   
magic = np.frombuffer(f.read(4), dtype=dt)[0]
if magic != 2051:
    raise ValueError('Invalid magic number %d in MNIST image file: %s' % (magic, f.name))
num_images = np.frombuffer(f.read(4), dtype=dt)[0]
rows = np.frombuffer(f.read(4), dtype=dt)[0]
cols = np.frombuffer(f.read(4), dtype=dt)[0]
buf = f.read(rows * cols * num_images)
data = np.frombuffer(buf, dtype=np.uint8)
test_images = data.reshape(num_images, rows, cols, 1)
f.close()    

In [ ]:
first_image = test_images[0].copy().reshape((28,28))
first_image = first_image.astype(np.uint8)
plt.imshow(first_image, cmap='Greys')

Carregar os rotulos (labels) de teste

In [ ]:
f = open('MNIST_data/t10k-labels.idx1-ubyte', 'rb')
magic = np.frombuffer(f.read(4), dtype=dt)[0]
if magic != 2049:
  raise ValueError('Invalid magic number %d in MNIST label file: %s' % (magic, f.name))
num_items = np.frombuffer(f.read(4), dtype=dt)[0]
buf = f.read(num_items)
labels = np.frombuffer(buf, dtype=np.uint8)
f.close()
num_labels = labels.shape[0]
index_offset = np.arange(num_labels) * 10
labels_one_hot = np.zeros((num_labels, 10))
labels_one_hot.flat[index_offset + labels.ravel()] = 1
test_labels = labels_one_hot

In [ ]:
test_labels

In [ ]:
validation_images = train_images[:5000]
validation_labels = train_labels[:5000]
train_images = train_images[5000:]
train_labels = train_labels[5000:]

Abaixo é alterado o formato da matriz de [num_imagens, 28, 28, 1] para [num_imagens, 728], isso para facilidade de trabalho pelo tensorflow.
É realizda também uma mudança nos pixel mudando de inteiro para float, onde cada pixel tem o seu valor alterado de uma variação de 0 a 255 para variação de 0 a 1.
Isso também por facilidade do tensorflow

In [ ]:
validation_images= validation_images.reshape(validation_images.shape[0], validation_images.shape[1] * validation_images.shape[2])
validation_images = validation_images.astype(np.float32)
validation_images = np.multiply(validation_images, 1.0 / 255.0)

train_images= train_images.reshape(train_images.shape[0], train_images.shape[1] * train_images.shape[2])
train_images = train_images.astype(np.float32)
train_images = np.multiply(train_images, 1.0 / 255.0)

test_images= test_images.reshape(test_images.shape[0], test_images.shape[1] * test_images.shape[2])
test_images = test_images.astype(np.float32)
test_images = np.multiply(test_images, 1.0 / 255.0)

Este é o resultado final das coletas dos arquivos.

In [ ]:
print(train_images)
print(train_labels)
print(test_images)
print(test_labels)
print(validation_images)
print(validation_labels)

## Implementação do tensor flow ##

In [ ]:
import tensorflow as tf

Define o parametro de entrada do modelo, neste ponto eu sei o tamanho da imagem (784), mas não sei quantas imagens serão incluidas.

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])

Variaveis de uso interno do tensorflow, serão utilizdas para calcular a expressão abaixo.
"W" tem a dimensão 784, 10 porque cada imagem irá gerar um indice de peso evidencia para cada classe, no caso temos 10 classes.

In [ ]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [ ]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

Parametro para indicar quais são os valores corretos para cada imagem.

In [ ]:
y_ = tf.placeholder(tf.float32, [None, 10])

Função para calculo da precisão do modelo. Ou função de distancia. Isso para gerar o melhor resultado possivel para o calculo.

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

Iniciar o treinamento

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

O codigo abaixo ajusta as imagens para o treinamento.
No primeiro laço é realizado uma permuta misturando as imagens, de forma que a acada analise um novo indice pode ser alcançado.
A partir deste ponto é coletado faixas de 100 imagens para analise.

In [ ]:
_epochs_completed = 0
_index_in_epoch = 0
for _ in range(1000):
    start = _index_in_epoch
    if _epochs_completed == 0 and start == 0:
        perm0 = np.arange(55000)
        np.random.shuffle(perm0)
        images = train_images[perm0]
        labels = train_labels[perm0]
    # Go to the next epoch
    _epochs_completed += 1
    print("Treinando... passo: {0:3}".format(_epochs_completed))
    if start + 100 > 55000:
        # Finished epoch
        # Get the rest examples in this epoch
        rest_num_examples = 55000 - start
        images_rest_part = images[start:55000]
        labels_rest_part = labels[start:55000]
        # Shuffle the data
        perm = np.arange(55000)
        np.random.shuffle(perm)
        images = train_images[perm]
        labels = train_labels[perm]
        # Start next epoch
        start = 0
        _index_in_epoch = 100 - rest_num_examples
        end = _index_in_epoch
        images_new_part = images[start:end]
        labels_new_part = labels[start:end]
        batch_xs = np.concatenate((images_rest_part, images_new_part), axis=0)
        batch_ys = np.concatenate((labels_rest_part, labels_new_part), axis=0)
    else:
        _index_in_epoch += 100
        end = _index_in_epoch
        batch_xs, batch_ys = images[start:end], labels[start:end]
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
print("Finalizado")

Verificando o Modelo

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
print(sess.run(accuracy, feed_dict={x: test_images, y_: test_labels}))